In [4]:
from numpy import array, sin, cos, tan
from numpy.linalg import norm
from lib.convert import deg2rad, rad2deg
from lib.rotation import Rx, Ry, Rz
from lib.geodesy import geod2ECEF, ECEF2geod


# System conditions
dt = 1                          # second
R = 6371e3                      # meter
oie_e = array([[0],
               [0],
               [7292115e-11]])  # radian/second (Earth rotation)

# Initial conditions
lat = deg2rad(45)               # radian
lon = -deg2rad(30)              # radian
h = 10000                       # meter

yaw = deg2rad(60)               # radian
pitch = 0                       # radian
roll  = 0                       # radian

veb_b = array([[800000/3600],   # meter/second
               [0],
               [0]])

# Observations
gyrox = -deg2rad(0.1)           # radian/second
gyroy = deg2rad(0.1)            # radian/second
gyroz = -deg2rad(1.0)           # radian/second

oib_b = array([[gyrox],         # radian/second
               [gyroy],
               [gyroz]])


# Local functions
# Create cross product matrix
def crossm(a):
    
    A = array([[ 0, -a[2, 0], a[1, 0]],
               [ a[2, 0], 0, -a[0, 0]],
               [-a[1, 0], a[0, 0], 0]])
    return A

# Normalize rotation matrix
def normalize(C):
    return (3/2)*C - (1/2)*C@C.T@C


# Compute Cb_g
Cb_g = Rz(yaw)@Ry(pitch)@Rx(roll)

# Compute Ce_g
Ce_g = array([[-sin(lat)*cos(lon), -sin(lat)*sin(lon), cos(lat)],
              [-sin(lon), cos(lon), 0],
              [-cos(lat)*cos(lon), -cos(lat)*sin(lon), -sin(lat)]])

# Compute velocity in g-frame
veb_g = Cb_g@veb_b

# Compute oeg_g
oeg_g = array([[ veb_g[1, 0]/(R+h)],
               [-veb_g[0, 0]/(R+h)],
               [-veb_g[1, 0]/(R+h)*tan(lat)]])

# Compute Cb_g_dot
ogb_b = oib_b - Cb_g.T@oeg_g - Cb_g.T@Ce_g@oie_e
Ogb_b = crossm(ogb_b)
Cb_g_dot = Cb_g@Ogb_b

# Estimate Cb_g
Cb_g_hat = Cb_g + Cb_g_dot*dt

# Normalize Cb_g_hat
Cb_g_hat = normalize(Cb_g_hat)

# Compute velocity in g-frame
veb_g = Cb_g_hat@veb_b

# Compute position increment in g-frame
dpeb_g = veb_g*dt

# Compute position in g-frame
lat = lat + dpeb_g[0, 0]/(R + h)
lon = lon + dpeb_g[1, 0]/((R + h)*cos(lat))
h = h - dpeb_g[2, 0]

print(rad2deg(lat))
print(rad2deg(lon))
print(h)

45.00096740630751
-30.00246806783165
10000.382176452787
